In [23]:
import os
import sys
import random

sys.path.append("../../../")
sys.path.append("SegFormer3D-main")


import yaml
import torch
import argparse
import numpy as np
from typing import Dict
from termcolor import colored
from accelerate import Accelerator
from losses.losses import build_loss_fn
from optimizers.optimizers import build_optimizer
from optimizers.schedulers import build_scheduler
from train_scripts.trainer_ddp import Segmentation_Trainer
from architectures.build_architecture import build_architecture
from dataloaders.build_dataset import build_dataset, build_dataloader
import warnings
warnings.filterwarnings("ignore")


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x14c68c54cc40>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 14c68c5877f0, raw_cell="import os
import sys
import random

sys.path.appen.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcnlogin22.science.ru.nl/home/ldokovic/aimi-project/SegFormer3D-main/experiments/uls_2023/eval_diceCE_experiment/tta_runner.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x14c68c54cc40>> (for post_run_cell), with arguments args (<ExecutionResult object at 14c68c5875e0, execution_count=23 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 14c68c5877f0, raw_cell="import os
import sys
import random

sys.path.appen.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcnlogin22.science.ru.nl/home/ldokovic/aimi-project/SegFormer3D-main/experiments/uls_2023/eval_diceCE_experiment/tta_runner.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [24]:
import wandb

wandb.login(key="440fbdfe19fc3547947869a7935dd2ad9028815b")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x14c68c54cc40>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 14c6682aa0b0, raw_cell="import wandb

wandb.login(key="440fbdfe19fc3547947.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcnlogin22.science.ru.nl/home/ldokovic/aimi-project/SegFormer3D-main/experiments/uls_2023/eval_diceCE_experiment/tta_runner.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x14c68c54cc40>> (for post_run_cell), with arguments args (<ExecutionResult object at 14c6682ab220, execution_count=24 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 14c6682aa0b0, raw_cell="import wandb

wandb.login(key="440fbdfe19fc3547947.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcnlogin22.science.ru.nl/home/ldokovic/aimi-project/SegFormer3D-main/experiments/uls_2023/eval_diceCE_experiment/tta_runner.ipynb#W1sdnNjb2RlLXJlbW90ZQ%3D%3D> result=True>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [25]:
%load_ext autoreload

%autoreload 2

##################################################################################################
def launch_experiment(config_path) -> Dict:
    """
    Builds Experiment
    Args:
        config (Dict): configuration file

    Returns:
        Dict: _description_
    """
    # load config
    config = load_config(config_path)

    # set seed
    seed_everything(config)

    if config["evaluate_only"]:
        print("[info] -- Running in evaluation mode only.")
        if not config["training_parameters"]["load_checkpoint"]["load_full_checkpoint"]:
            raise ValueError("Checkpoint path must be provided for evaluation.")
    else:
        # build directories
        build_directories(config)

    # build training dataset & training data loader
    trainset = build_dataset(
        dataset_type=config["dataset_parameters"]["dataset_type"],
        dataset_args=config["dataset_parameters"]["train_dataset_args"],
    )
    trainloader = build_dataloader(
        dataset=trainset,
        dataloader_args=config["dataset_parameters"]["train_dataloader_args"],
        config=config,
        train=True,
    )

    # build validation dataset & validataion data loader
    valset = build_dataset(
        dataset_type=config["dataset_parameters"]["dataset_type"],
        dataset_args=config["dataset_parameters"]["val_dataset_args"],
    )
    valloader = build_dataloader(
        dataset=valset,
        dataloader_args=config["dataset_parameters"]["val_dataloader_args"],
        config=config,
        train=False,
    )

    # build the Model
    model = build_architecture(config)

    # set up the loss function
    criterion = build_loss_fn(
        loss_type=config["loss_fn"]["loss_type"],
        loss_args=config["loss_fn"]["loss_args"],
    )

    # set up the optimizer
    optimizer = build_optimizer(
        model=model,
        optimizer_type=config["optimizer"]["optimizer_type"],
        optimizer_args=config["optimizer"]["optimizer_args"],
    )

    # set up schedulers
    warmup_scheduler = build_scheduler(
        optimizer=optimizer, scheduler_type="warmup_scheduler", config=config
    )
    training_scheduler = build_scheduler(
        optimizer=optimizer,
        scheduler_type="training_scheduler",
        config=config,
    )

    # use accelarate
    accelerator = Accelerator(
        log_with="wandb",
        gradient_accumulation_steps=config["training_parameters"][
            "grad_accumulate_steps"
        ],
    )
    accelerator.init_trackers(
        project_name=config["project"],
        config=config,
        init_kwargs={"wandb": config["wandb_parameters"]},
    )

    # display experiment info
    display_info(config, accelerator, trainset, valset, model)

    # convert all components to accelerate
    model = accelerator.prepare_model(model=model)
    optimizer = accelerator.prepare_optimizer(optimizer=optimizer)
    trainloader = accelerator.prepare_data_loader(data_loader=trainloader)
    valloader = accelerator.prepare_data_loader(data_loader=valloader)
    warmup_scheduler = accelerator.prepare_scheduler(scheduler=warmup_scheduler)
    training_scheduler = accelerator.prepare_scheduler(scheduler=training_scheduler)

        

    # create a single dict to hold all parameters
    storage = {
        "model": model,
        "trainloader": trainloader,
        "valloader": valloader,
        "criterion": criterion,
        "optimizer": optimizer,
        "warmup_scheduler": warmup_scheduler,
        "training_scheduler": training_scheduler,
    }

    # set up trainer
    trainer = Segmentation_Trainer(
        config=config,
        model=storage["model"],
        optimizer=storage["optimizer"],
        criterion=storage["criterion"],
        train_dataloader=storage["trainloader"],
        val_dataloader=storage["valloader"],
        warmup_scheduler=storage["warmup_scheduler"],
        training_scheduler=storage["training_scheduler"],
        accelerator=accelerator,
    )

    if config["training_parameters"]["load_checkpoint"]["load_full_checkpoint"]:
        print("[info] -- Loading checkpoint.")
        load_checkpoint(
            config=config,
            accelerator=accelerator,
            storage=storage,
            trainer=trainer,)


    print("[info] -- Setup complete.")

    # run train
    if config["evaluate_only"]:
        print("[info] -- Running evaluation only.")
        trainer.evaluate()
    else:
        trainer.train()


##################################################################################################
def seed_everything(config) -> None:
    seed = config["training_parameters"]["seed"]
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


##################################################################################################
def load_config(config_path: str) -> Dict:
    """loads the yaml config file

    Args:
        config_path (str): _description_

    Returns:
        Dict: _description_
    """
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)
    return config


##################################################################################################
def build_directories(config: Dict) -> None:
    # create necessary directories
    if not os.path.exists(config["training_parameters"]["checkpoint_save_dir"]):
        os.makedirs(config["training_parameters"]["checkpoint_save_dir"])
    
    last_model_dir = os.path.join(
        config["training_parameters"]["checkpoint_save_dir"],
        "last_epoch_model",
    )
    if not os.path.exists(last_model_dir):
        os.makedirs(last_model_dir)


    if os.listdir(config["training_parameters"]["checkpoint_save_dir"]) and \
        not config["training_parameters"]["load_checkpoint"]["load_full_checkpoint"]:
        raise ValueError("checkpoint exits -- preventing file override -- rename file")


##################################################################################################
def display_info(config, accelerator, trainset, valset, model):
    # print experiment info
    accelerator.print(f"-------------------------------------------------------")
    accelerator.print(f"[info]: Experiment Info")
    accelerator.print(
        f"[info] ----- Project: {colored(config['project'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Group: {colored(config['wandb_parameters']['group'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Name: {colored(config['wandb_parameters']['name'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Batch Size: {colored(config['dataset_parameters']['train_dataloader_args']['batch_size'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Num Epochs: {colored(config['training_parameters']['num_epochs'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Loss: {colored(config['loss_fn']['loss_type'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Optimizer: {colored(config['optimizer']['optimizer_type'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Train Dataset Size: {colored(len(trainset), color='red')}"
    )
    accelerator.print(
        f"[info] ----- Test Dataset Size: {colored(len(valset), color='red')}"
    )

    pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    accelerator.print(
        f"[info] ----- Distributed Training: {colored('True' if torch.cuda.device_count() > 1 else 'False', color='red')}"
    )
    accelerator.print(
        f"[info] ----- Num Clases: {colored(config['model_parameters']['num_classes'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- EMA: {colored(config['ema']['enabled'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Load From Checkpoint: {colored(config['training_parameters']['load_checkpoint']['load_full_checkpoint'], color='red')}"
    )
    accelerator.print(
        f"[info] ----- Params: {colored(pytorch_total_params, color='red')}"
    )
    accelerator.print(f"-------------------------------------------------------")


###################################################################################################

def load_checkpoint(
        config,
        accelerator,
        storage,
        trainer        
):
    accelerator.load_state(
        config["training_parameters"]["load_checkpoint"]["load_checkpoint_path"]
    )
    # load trainer state
    trainer_state_dict = torch.load(
        os.path.join(config["training_parameters"]["load_checkpoint"]["load_checkpoint_path"],
        "trainer_state_dict.pkl"),
    )
    for key, value in trainer_state_dict.items():
        if hasattr(trainer, key):
            setattr(trainer, key, value)
        else:
            print(f"Key {key} not found in trainer state dict")
        
    # load current scheduler
    if trainer_state_dict["current_epoch"] > config["warmup_scheduler"]["warmup_epochs"]:
        trainer.scheduler = storage["training_scheduler"]
    else:
        trainer.scheduler = storage["warmup_scheduler"]

##################################################################################################

# parser = argparse.ArgumentParser(description="Simple example of training script.")
# parser.add_argument(
#     "--config", type=str, default="config.yaml", help="path to yaml config file"
# )
# args = parser.parse_args()
os.chdir('/home/ldokovic/aimi-project/SegFormer3D-main/experiments/uls_2023/eval_diceCE_experiment')
launch_experiment('./config.yaml')


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x14c68c54cc40>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 14c626331960, raw_cell="%load_ext autoreload

%autoreload 2

#############.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bcnlogin22.science.ru.nl/home/ldokovic/aimi-project/SegFormer3D-main/experiments/uls_2023/eval_diceCE_experiment/tta_runner.ipynb#W2sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[info] -- Running in evaluation mode only.
Initializing dataset with csv file:  ../../../data/uls2023_seg/train.csv
Number of samples:  4907
Loaded ULS2023 dataset
Initializing dataset with csv file:  ../../../data/uls2023_seg/validation.csv
Number of samples:  866
Loaded ULS2023 dataset


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


-------------------------------------------------------
[info]: Experiment Info
[info] ----- Project: uls2023
[info] ----- Group: first_experiment
[info] ----- Name: segformer3d_adamw_batch8_diceCEloss_continued
[info] ----- Batch Size: 8
[info] ----- Num Epochs: 400
[info] ----- Loss: diceCE
[info] ----- Optimizer: adamw
[info] ----- Train Dataset Size: 4907
[info] ----- Test Dataset Size: 866
[info] ----- Distributed Training: False
[info] ----- Num Clases: 2
[info] ----- EMA: False
[info] ----- Load From Checkpoint: True
[info] ----- Params: 4492066
-------------------------------------------------------
[info] -- Trainer initialized!
[info] -- Loading checkpoint.
[info] -- Setup complete.
[info] -- Running evaluation only.
[info] -- starting evaluation
[info] -- Starting model evaluation
YESS


100%|██████████| 866/866 [02:26<00:00,  5.91it/s]

eval loss -- 0.00000 || eval mean_uls_metric -- 0.00000 -- savedeval my_mean_uls_metric -- 0.00000 -- savedeval mean_dice -- 0.61598 -- saved


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
